# Read Data from SQLite and Create Excel & PDF

In [1]:
import sqlite3
import sys
import xlsxwriter
import fpdf
from fpdf import FPDF

In [2]:
class Data_import_export:
    def __init__(self):
        pass
    
    #get the table from sqlite
    def get_table_from_sqlite(self, db_name):
        connect_sqlite = sqlite3.connect(db_name)
        cursor_sqlite = connect_sqlite.cursor()
        SELECT= "SELECT name FROM sqlite_master WHERE type='table'"
        
        #read tables 
        cursor_sqlite.execute(SELECT)
        results=cursor_sqlite.fetchall()
        data=[]
        
        #displaying all table names in the database
        for row in results:
            data.append(row[0])
        if data:
            return data
        else:
            return "Database does not exist or there is no table in database"
    
    #get fields/columns from the table
    def get_fields_from_sqlite(self, db_name, table_name):
        connect_sqlite=sqlite3.connect(db_name)
        cursor_sqlite=connect_sqlite.cursor()
        select_column='SELECT DISTINCT * FROM '+ table_name + ';'
        
        table=cursor_sqlite.execute(select_column)
        list_columns=list(map(lambda x: x[0], table.description))
        
        return list_columns
    
    #after we get the table and columns, we need to fillin the data to the table 
    def get_data_from_sqlite(self, db_name, table_name,fields):
        connect_sqlite=sqlite3.connect(db_name)
        cursor_sqlite=connect_sqlite.cursor()
        select_data='SELECT DISTINCT '+ fields+ ' from ' + table_name +';'
        cursor_sqlite.execute(select_data)
        result=cursor_sqlite.fetchall()
        return result
    
    
    def export_table_to_excel(self, table_name, fields, data):
        workbook=xlsxwriter.Workbook(table_name +'.xlsx')
        worksheet = workbook.add_worksheet()
        
        #writing the colnames, starting from the first row
        row=0
        col=0
        for value in fields:
            worksheet.write(row, col, value)
            col+=1
        
        #writing the data row by row, starting from the second row (row=1)
        row=1
        col=0
        
        for row1 in data:
            col=0
            for index in range(len(row1)):
                worksheet.write(row, col, row1[index])
                col+=1
            row+=1
        
        workbook.close()
        return 'done'
    
    def export_table_to_pdf(self, table_name, fields, data):
        print(fields)
        print(data)
        pdf=FPDF()
        array=list()
        for i in range(len(fields)):
            array.append(30)
        
        width_cell=tuple(array)
        
        #add a page
        pdf.add_page()
        pdf.set_font('Arial', 'B', 20)
        pdf.cell(200,10, "Employee Information", 0,1,'C')
        
        pdf.set_font('Arial', size=12)
        pdf.cell(80,10,table_name,0,1,'C')
        
        for i in range(len(fields)):
            if i ==len(fields)-1:
                pdf.cell(width_cell[i], 10, fields[i], 1, 1, "C", "true")
            else:
                pdf.cell(width_cell[i], 10, fields[i], 1, 0, "C", "true")
        
        print(data)
        
        for row in data:
            print(row)
            for i in range(len(row)):
                if i==len(row)-1:
                    pdf.cell(width_cell[i], 10, str(row[i]),1,1,"C", "true")
                else:
                    pdf.cell(width_cell[i], 10, str(row[i]),1,0,"C", "true")
                    
        pdf.set_y(-50)
        pdf.set_font('Arial', "I", 8)
        pdf.cell(200,10,"Page # " + str(pdf.page_no()), 0, 1, 'C')
        pdf.output(table_name+".pdf")
        return 'done'
    
    def export_data_to_txt(self, table_name, fields, data):
        file=open(table_name+'.txt', "w")
        file.writelines(fields)
        file.writelines(data)
        file.close()

In [3]:
#testing the methods
connection=sqlite3.connect("company.db")
cursor=connection.cursor()

In [4]:
sql_command = """
CREATE TABLE employee ( 
staff_number INTEGER PRIMARY KEY, 
fname VARCHAR(20), 
lname VARCHAR(30), 
gender CHAR(1), 
joining DATE,
birth_date DATE);"""

#cursor.execute(sql_command)

sql_command = """INSERT INTO employee (staff_number, fname, lname, gender, birth_date)
    VALUES (NULL, "William", "Shakespeare", "m", "1961-10-25");"""
#cursor.execute(sql_command)

sql_command = """INSERT INTO employee (staff_number, fname, lname, gender, birth_date)
    VALUES (NULL, "Frank", "Schiller", "m", "1955-08-17");"""
#cursor.execute(sql_command)

connection.commit()

In [5]:
staff_data = [ ("William", "Shakespeare", "m", "1961-10-25"),
               ("Frank", "Schiller", "m", "1955-08-17"),
               ("Jane", "Wall", "f", "1989-03-14") ]
               
for p in staff_data:
    format_str = """INSERT INTO employee (staff_number, fname, lname, gender, birth_date)
    VALUES (NULL, "{first}", "{last}", "{gender}", "{birthdate}");"""

    sql_command = format_str.format(first=p[0], last=p[1], gender=p[2], birthdate = p[3])
    #cursor.execute(sql_command)

connection.commit()

In [6]:
cursor.execute("SELECT * FROM employee") 
print("fetchall:")
result = cursor.fetchall() 
for r in result:
    print(r)

fetchall:
(1, 'William', 'Shakespeare', 'm', None, '1961-10-25')
(2, 'Frank', 'Schiller', 'm', None, '1955-08-17')
(3, 'William', 'Shakespeare', 'm', None, '1961-10-25')
(4, 'Frank', 'Schiller', 'm', None, '1955-08-17')
(5, 'Jane', 'Wall', 'f', None, '1989-03-14')
(6, 'William', 'Shakespeare', 'm', None, '1961-10-25')
(7, 'Frank', 'Schiller', 'm', None, '1955-08-17')
(8, 'Jane', 'Wall', 'f', None, '1989-03-14')
(9, 'William', 'Shakespeare', 'm', None, '1961-10-25')
(10, 'Frank', 'Schiller', 'm', None, '1955-08-17')
(11, 'William', 'Shakespeare', 'm', None, '1961-10-25')
(12, 'Frank', 'Schiller', 'm', None, '1955-08-17')
(13, 'Jane', 'Wall', 'f', None, '1989-03-14')
(14, 'William', 'Shakespeare', 'm', None, '1961-10-25')
(15, 'Frank', 'Schiller', 'm', None, '1955-08-17')
(16, 'Jane', 'Wall', 'f', None, '1989-03-14')


In [7]:
import_n_export = Data_import_export()
import_n_export.get_table_from_sqlite('company.db')

['employee']

In [8]:
import_n_export.get_fields_from_sqlite('company.db', 'employee')

['staff_number', 'fname', 'lname', 'gender', 'joining', 'birth_date']

In [9]:
fields=import_n_export.get_fields_from_sqlite('company.db', 'employee')
import_n_export.get_data_from_sqlite('company.db', 'employee', "*")

[(1, 'William', 'Shakespeare', 'm', None, '1961-10-25'),
 (2, 'Frank', 'Schiller', 'm', None, '1955-08-17'),
 (3, 'William', 'Shakespeare', 'm', None, '1961-10-25'),
 (4, 'Frank', 'Schiller', 'm', None, '1955-08-17'),
 (5, 'Jane', 'Wall', 'f', None, '1989-03-14'),
 (6, 'William', 'Shakespeare', 'm', None, '1961-10-25'),
 (7, 'Frank', 'Schiller', 'm', None, '1955-08-17'),
 (8, 'Jane', 'Wall', 'f', None, '1989-03-14'),
 (9, 'William', 'Shakespeare', 'm', None, '1961-10-25'),
 (10, 'Frank', 'Schiller', 'm', None, '1955-08-17'),
 (11, 'William', 'Shakespeare', 'm', None, '1961-10-25'),
 (12, 'Frank', 'Schiller', 'm', None, '1955-08-17'),
 (13, 'Jane', 'Wall', 'f', None, '1989-03-14'),
 (14, 'William', 'Shakespeare', 'm', None, '1961-10-25'),
 (15, 'Frank', 'Schiller', 'm', None, '1955-08-17'),
 (16, 'Jane', 'Wall', 'f', None, '1989-03-14')]

In [10]:
table_name = import_n_export.get_table_from_sqlite('company.db')
fields = import_n_export.get_fields_from_sqlite('company.db', 'employee')
data = import_n_export.get_data_from_sqlite('company.db', 'employee', "*")
import_n_export.export_table_to_excel('employee', fields, data)

'done'

In [11]:
#export data to PDF
import_n_export.export_table_to_pdf('employee', fields, data)

['staff_number', 'fname', 'lname', 'gender', 'joining', 'birth_date']
[(1, 'William', 'Shakespeare', 'm', None, '1961-10-25'), (2, 'Frank', 'Schiller', 'm', None, '1955-08-17'), (3, 'William', 'Shakespeare', 'm', None, '1961-10-25'), (4, 'Frank', 'Schiller', 'm', None, '1955-08-17'), (5, 'Jane', 'Wall', 'f', None, '1989-03-14'), (6, 'William', 'Shakespeare', 'm', None, '1961-10-25'), (7, 'Frank', 'Schiller', 'm', None, '1955-08-17'), (8, 'Jane', 'Wall', 'f', None, '1989-03-14'), (9, 'William', 'Shakespeare', 'm', None, '1961-10-25'), (10, 'Frank', 'Schiller', 'm', None, '1955-08-17'), (11, 'William', 'Shakespeare', 'm', None, '1961-10-25'), (12, 'Frank', 'Schiller', 'm', None, '1955-08-17'), (13, 'Jane', 'Wall', 'f', None, '1989-03-14'), (14, 'William', 'Shakespeare', 'm', None, '1961-10-25'), (15, 'Frank', 'Schiller', 'm', None, '1955-08-17'), (16, 'Jane', 'Wall', 'f', None, '1989-03-14')]
[(1, 'William', 'Shakespeare', 'm', None, '1961-10-25'), (2, 'Frank', 'Schiller', 'm', None, '19

'done'

In [12]:
import_n_export.export_data_to_txt('employee', fields, data)

TypeError: write() argument must be str, not tuple